## Imports

In [14]:
import pandas as pd
import numpy as np

import nltk

import re

import spacy

#### Reading the File

In [3]:
!pip install xlrd

In [4]:
df = pd.read_excel('Documents.xlsx')

#### managing twits lengths

In [5]:
df['length'] = [len(x) for x in df['Title']]
max_length = int(np.max(df['length']))

In [6]:
max_length

536

## Preprocessing

#### Managing hyperlinks

In [7]:
def get_hyperlinks(text):
    ## finds all hyperlinks in the text
    return re.findall(r"http\S+", text)

def rmv_hyperlink(text):
    ## removes all hyperlinks of the text
    return re.sub(r"http\S+", "", text)


df['hyperlinks'] = df['Title'].map(
    get_hyperlinks
)

df['Title'] = df['Title'].map(
    rmv_hyperlink
)

#### Managing hashtags

In [8]:
def get_hashtags(text):
    ## finds all hashtags of the text
    return re.findall(r"#\S+", text)

def rmv_hashtags(text):
    ## removes all hashtags of the text
    return re.sub(r"#\S+", "", text)

In [9]:
df['hashtags'] = df['Title'].map(
    get_hashtags
)

df['Title'] = df['Title'].map(
    rmv_hashtags
)

#### Managing user references '@'

In [10]:
def get_users(text):
    return re.findall(r"@\S+", text)

In [11]:
df['referenced_users'] = df['Title'].map(
    get_users
)

#### tokenizing in sentences and PoS

In [16]:
## french PoS:
## https://github.com/cmchurch/nltk_french/blob/master/french-nltk.py

## https://nlp.stanford.edu/software/CRF-NER.html

nlp = spacy.load('fr_core_news_sm')

df['Title'].map(
    nltk.sent_tokenize
)

0              [Il a guéri du COVID ou il a tué Thanos ?]
1       [les premiers symptômes du coronavirus c’est l...
2       [Je souhaite une agréable journée à tout le mo...
3                 [Le COVID va bientôt fêter ses 1ans la]
4       [Le covid il a passé plus de temps que moi à l...
                              ...                        
9995    [[ Le taux de positivité chez les patients sym...
9996    [La @VilledeGrenoble, via le CCAS, a distribué...
9997    [Considérée comme un "cas contact", Roselyne B...
9998    [Bonjour, test covid très douloureux ce matin....
9999    [HAHA\n\nUne main apparaît dans le champ de vi...
Name: Title, Length: 10000, dtype: object

In [25]:
sample_text = """Mais ces derniers sont loin d’exclure l’idée que le philosophe s’intéresse à des problèmes théoriques. La « sagesse », ou plus exactement la sophia, que souhaite appréhender le philosophe est aussi un savoir et une connaissance. Le philosophe, dans la lignée de la tradition fondée par Socrate, sait comment il doit vivre ; il peut justifier ses choix et son mode de vie. Socrate par exemple, dans les dialogues socratiques de Platon, exige de ses interlocuteurs qu’ils soient à même de donner le logos de leur jugement de valeur et de leur choix, c’est-à-dire de les justifier rationnellement. Cette exigence de rationalité peut même amener à donner des fondements authentiquement scientifiques à la philosophie. Bien sûr la définition de la philosophie en tant que modus vivendi (mode de vie) ne peut prétendre être suffisante pour définir la philosophie dans son ensemble. Bien des philosophes ont compris la philosophie comme un travail intellectuel et non comme un mode de vie : c'est le cas dans le monde universitaire et de la recherche de nos jours. Il en va tout autrement, en Inde notamment. Le point de vue occidental ne peut s'appliquer aux concepts philosophiques en vigueur dans cette partie du monde, bien qu'il y eût tentative d'assimilation à l'époque romaine, en particulier avec Plotin."""

docs = [nlp(s) for s in nltk.sent_tokenize(sample_text)]

for (i,doc) in enumerate(docs):
    print('--{}--'.format(i))
    print(doc)
    for entity in doc.ents:
        print('entity: {} || entity label: {}'.format(entity, entity.label_))

--0--
Mais ces derniers sont loin d’exclure l’idée que le philosophe s’intéresse à des problèmes théoriques.
--1--
La « sagesse », ou plus exactement la sophia, que souhaite appréhender le philosophe est aussi un savoir et une connaissance.
entity: » || entity label: MISC
--2--
Le philosophe, dans la lignée de la tradition fondée par Socrate, sait comment il doit vivre ; il peut justifier ses choix et son mode de vie.
entity: Socrate || entity label: PER
--3--
Socrate par exemple, dans les dialogues socratiques de Platon, exige de ses interlocuteurs qu’ils soient à même de donner le logos de leur jugement de valeur et de leur choix, c’est-à-dire de les justifier rationnellement.
entity: Socrate || entity label: PER
entity: Platon || entity label: PER
--4--
Cette exigence de rationalité peut même amener à donner des fondements authentiquement scientifiques à la philosophie.
--5--
Bien sûr la définition de la philosophie en tant que modus vivendi (mode de vie) ne peut prétendre être suff

### Creating vocabulary

#### Tokenizing and Removing stopwords

In [44]:
from nltk.corpus import stopwords

stopwords = stopwords.words('french')

def rmv_stop_word(tokens):
    return [x.lower() for x in tokens if x.lower() not in stopwords]

df['Title'] = df['Title'].map(
    nltk.word_tokenize
).map(
    rmv_stop_word
)

#### Stemming

In [45]:
## https://stackoverflow.com/questions/13131139/lemmatize-french-text

from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()

def stem(tokens):
    return [stemmer.stem(x) for x in tokens]

df['Title'] = df['Title'].map(
    stem
)

In [46]:
df['Title']

0                      [a, guer, covid, a, tu, thanos, ?]
1       [premi, symptôm, coronavirus, ’, pert, goût, h...
2       [souhait, agréabl, journ, tout, mond, tout, mo...
3                          [covid, va, bientôt, fêt, 1an]
4                  [covid, a, pass, plus, temp, fac, con]
                              ...                        
9995    [[, #, covid19, ], taux, posit, chez, patient,...
9996    [@, villedegrenobl, ,, vi, ccas, ,, a, distrib...
9997    [consider, comm, ``, cas, contact, '', ,, rose...
9998    [bonjour, ,, test, covid, tres, doulour, matin...
9999    [hah, main, apparaît, champ, vision, d'alex, ,...
Name: Title, Length: 10000, dtype: object